In [ ]:
from bs4 import BeautifulSoup
import requests
import os
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'http://books.toscrape.com/'
browser.visit(url)

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_medical_abbreviations'
med_abbreviations = ['BMR', 'BP', 'ECG', 'MRI', 'qid', 'WBC']

In [ ]:
#We can use the read_html function in Pandas to automatically scrape any tabular data from a page (tables).
tables = pd.read_html(url)
tables

In [ ]:
#What we get in return is a list of dataframes for any tabular data that Pandas found.
type(tables)

In [ ]:
#We can slice off any of those dataframes that we want using normal indexing.
df = tables[]
df.columns = ['abb', 'full_name', 'other']
df.head

In [ ]:
# Cleanup of extra row
del df['other']

In [ ]:
# Drop all single header rows
df.columns = df.columns.get_level_values(0)
df = df.loc[df.Ref.str.startswith("[")]
df.head()

In [ ]:
# Drop the first row and set the index to the `abb` column
### BEGIN SOLUTION
df = df.iloc[1:]
df.set_index('abb', inplace=True)
df.head()
### END SOLUTION

In [ ]:
# Split column values into two separate columns
columnsplit = df['City'].str.split(", ", expand=True)
df = df.assign(City=columnsplit[0],State=columnsplit[1])
df.head()

In [ ]:
# Drop a column
df = df.drop(['Ref'], axis=1)
df.head()

In [ ]:
# Reset an index
df = df.reset_index(drop=True)
df.head()

In [ ]:
 df.loc[df.State=="New York"]

In [ ]:
 # Loop through the list of medical abbreviations and print the abbreviation
# along with the full description.
# Use the DataFrame to perform the lookup.
### BEGIN SOLUTION
for abb in med_abbreviations:
    print(abb, df.loc[abb].full_name)
### END SOLUTION

In [ ]:
 # Iterate through all pages
for x in range(50):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
    articles = soup.find_all('article', class_='product_pod')

    # Iterate through each book
    for article in articles:
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
        h3 = article.find('h3')
        link = h3.find('a')
        href = link['href']
        title = link['title']
        print('-----------')
        print(title)
        print('http://books.toscrape.com/' + href)

    # Click the 'Next' button on each page
    try:
        browser.links.find_by_partial_text('next').click()
          
    except:
        print("Scraping Complete")

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

sidebar = soup.find('ul', class_='nav-list')

categories = sidebar.find_all('li')

category_list = []
url_list = []
book_url_list = []
for category in categories:
    title = category.text.strip()
    category_list.append(title)
    book_url = category.find('a')['href']
    url_list.append(book_url)

book_url_list = ['http://books.toscrape.com/' + url for url in url_list]

titles_and_urls = zip(category_list, book_url_list)

try:
    for title_url in titles_and_urls:
        browser.links.find_by_partial_text('next').click()
except ElementDoesNotExist:
    print("Scraping Complete")

In [ ]:
book_url_list

In [ ]:
browser.quit()

In [ ]:
# Pandas also had a to_html method that we can use to generate HTML tables from DataFrames.
html_table = df.to_html()
html_table

In [ ]:
# You may have to strip unwanted newlines to clean up the table.
html_table.replace('\n', '')   

In [ ]:
# You can also save the table directly to a file.
df.to_html('table.html')